In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pre_processing import *

import os

from sklearn.model_selection import train_test_split, GridSearchCV

from scipy.signal import spectrogram

import torch
import torch.nn.init as init
from torch.nn import Module, Conv2d, Linear, MaxPool2d, ReLU, LogSoftmax, Flatten, Dropout, Sigmoid, BCELoss
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
torch.manual_seed(42)
from dataclasses import dataclass

import skorch
from ray import tune
from ray.tune.schedulers import ASHAScheduler


from PIL import Image
from skimage.io import imread

from bayes_opt import BayesianOptimization

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
class make_data(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.dataset = datasets.ImageFolder(self.data_dir, transform=self.transform)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        return self.dataset[index]

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((240, 320))])
dataset = make_data('pic_data/spectro/full_prsb/', transform=transform)

In [4]:
train_data, test_data = random_split(dataset, lengths=[0.8, 0.2])

In [5]:
train_loader = DataLoader(train_data, batch_size=16)
test_loader = DataLoader(test_data, batch_size=16)

In [6]:
# Checking Sizes of Layers
conv1 = Conv2d(3,64,3)
pool = MaxPool2d(2,2)
conv2 = Conv2d(64,64,5)
img_conv1 = conv1(dataset[0][0])
img_pool1 = pool(img_conv1)
img_conv2 = conv2(img_pool1)
img_pool2 = pool(img_conv2)
img_conv3 = conv2(img_pool2)
img_pool3 = pool(img_conv3)

print(f'{img_conv1.shape=}')
print(f'{img_pool1.shape=}')
print(f'{img_conv2.shape=}')
print(f'{img_pool2.shape=}')
print(f'{img_conv3.shape=}')
print(f'{img_pool3.shape=}')

img_conv1.shape=torch.Size([64, 238, 318])
img_pool1.shape=torch.Size([64, 119, 159])
img_conv2.shape=torch.Size([64, 115, 155])
img_pool2.shape=torch.Size([64, 57, 77])
img_conv3.shape=torch.Size([64, 53, 73])
img_pool3.shape=torch.Size([64, 26, 36])


In [17]:
class cnn_maker(Module):
    def __init__(self):
        super(cnn_maker, self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.pool = MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.conv3 = Conv2d(in_channels=64, out_channels=64, kernel_size=5)
        self.fc1 = Linear(64 * 26 * 36, 128)
        self.fc2 = Linear(128, 64)
        self.fc3 = Linear(64, 32)
        self.drop = Dropout()
        self.fc_f = Linear(32, 1)
        self.prob = Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64* 26 * 36)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        
        x = self.prob(self.fc_f(x))
        return x

In [8]:
model = skorch.NeuralNetClassifier(cnn_maker, criterion = torch.nn.BCELoss, 
                                   optimizer = optim.SGD, max_epochs = 100, batch_size = 10)
model.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=cnn_maker(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (conv3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (fc1): Linear(in_features=59904, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=32, bias=True)
    (drop): Dropout(p=0.5, inplace=False)
    (fc_f): Linear(in_features=32, out_features=1, bias=True)
    (prob): Sigmoid()
  ),
)

In [9]:
x = [] 
y = []
for i in train_loader:
    try:
        x.append(np.array(i[0]))
        y.append(np.array(i[1]))
    except:
        continue

In [10]:
x_full = np.concatenate(x, axis = 0)
y_full = np.concatenate(y, axis = 0)

# Batch size

In [12]:
param_grid = {
    'batch_size': [10, 20, 40, 60, 80, 100],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=3)
grid_result = grid.fit(x_full[:100].astype(np.float32), y_full[:100].reshape(-1, 1).astype(np.float32))
 
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6827       0.6000        0.6805  3.9599
      2        0.6862       0.6000        0.6800  4.0121
      3        0.6808       0.6000        0.6797  4.0045
      4        0.6870       0.6000        0.6793  4.0501
      5        0.6792       0.6000        0.6789  4.1109
      6        0.6830       0.6000        0.6785  4.1856
      7        0.6812       0.6000        0.6781  4.1748
      8        0.6797       0.6000        0.6778  4.3423
      9        0.6763       0.6000        0.6776  4.2328
     10        0.6796       0.6000        0.6774  4.1020
     11        0.6816       0.6000        0.6772  4.0365
     12        0.6814       0.6000        0.6769  3.9766
     13        0.6809       0.6000        0.6767  3.9776
     14        0.6784       0.6000        0.6765  4.0129
     15        0.6750       0.6000        0.6762  3.9475
     16        0.6759       0.6

# Batch size: 10

# Optimiser

In [18]:
model = skorch.NeuralNetClassifier(cnn_maker, criterion = torch.nn.BCELoss, max_epochs = 50, batch_size = 10)
model.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=cnn_maker(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (conv3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (fc1): Linear(in_features=59904, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=32, bias=True)
    (drop): Dropout(p=0.5, inplace=False)
    (fc_f): Linear(in_features=32, out_features=1, bias=True)
    (prob): Sigmoid()
  ),
)

In [19]:
param_grid = {
    'optimizer': [optim.SGD, optim.RMSprop, optim.Adagrad, optim.Adadelta,
                  optim.Adam, optim.Adamax, optim.NAdam],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=3)
grid_result = grid.fit(x_full[:100].astype(np.float32), y_full[:100].reshape(-1, 1).astype(np.float32))
 
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        5.9017       0.6000        0.9599  4.2215
      2        0.9051       0.6000        0.6647  4.2818
      3        0.6919       0.6000        0.6697  4.3282
      4        0.6959       0.6000        0.6770  4.1937
      5        0.6744       0.6000        0.6434  4.4311
      6        0.6943       0.6000        0.6424  4.1739
      7        0.6791       0.6000        0.6126  4.3571
      8        0.6397       0.6000        0.5973  4.2576
      9        0.6779       0.6000        0.6021  4.2726
     10        0.6554       0.6000        0.6307  4.3388
     11        0.6602       0.6000        0.6086  4.2070
     12        0.6339       0.6500        0.5820  4.4245
     13        0.6274       0.7000        0.5940  4.1756
     14        0.6373       0.7000        0.6228  4.3717
     15        0.6042       0.7000        0.6070  4.3212
     16        0.5762       0.7

# Best: 0.719846 using {'optimizer': <class 'torch.optim.adagrad.Adagrad'>}

# lr 

In [21]:
model = skorch.NeuralNetClassifier(cnn_maker, criterion = torch.nn.BCELoss, optimizer = optim.Adagrad,
                                   max_epochs = 50, batch_size = 10)
model.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=cnn_maker(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (conv3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (fc1): Linear(in_features=59904, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=32, bias=True)
    (drop): Dropout(p=0.5, inplace=False)
    (fc_f): Linear(in_features=32, out_features=1, bias=True)
    (prob): Sigmoid()
  ),
)

In [23]:
param_grid = {
    'optimizer__lr': [0.001, 0.01, 0.1, 0.2, 0.3]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=3)
grid_result = grid.fit(x_full[:100].astype(np.float32), y_full[:100].reshape(-1, 1).astype(np.float32))
 
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        7.7633       0.6000        0.6768  4.2055
      2        0.6957       0.6000        0.6694  4.1300
      3        0.6875       0.6000        0.6725  4.1885
      4        0.6868       0.6000        0.6606  4.1430
      5        0.6990       0.6000        0.6635  4.1536
      6        0.6665       0.6000        0.6575  4.1545
      7        0.6809       0.6000        0.6555  4.1091
      8        0.6742       0.6000        0.6461  4.2699
      9        0.6724       0.6000        0.6206  4.2534
     10        0.6421       0.6000        1.0991  4.2192
     11        0.8901       0.6000        0.6366  4.1304
     12        0.6569       0.6000        0.5959  4.2322
     13        0.6761       0.6000        0.6084  4.1817
     14        0.6697       0.6000        0.5993  4.1340
     15        0.6516       0.6000        0.6117  4.1981
     16        0.6405       0.7

# Best: 0.719548 using {'optimizer__lr': 0.01}

# Weight init

In [25]:
class cnn_maker(Module):
    def __init__(self, weight_init=torch.nn.init.xavier_uniform_):
        super(cnn_maker, self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.pool = MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.conv3 = Conv2d(in_channels=64, out_channels=64, kernel_size=5)
        self.fc1 = Linear(64 * 26 * 36, 128)
        self.fc2 = Linear(128, 64)
        self.fc3 = Linear(64, 32)
        self.drop = Dropout()
        self.fc_f = Linear(32, 1)
        self.prob = Sigmoid()
        
        weight_init(self.fc1.weight)
        weight_init(self.fc_f.weight)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64* 26 * 36)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        
        x = self.prob(self.fc_f(x))
        return x

In [27]:
model = skorch.NeuralNetClassifier(cnn_maker, criterion = torch.nn.BCELoss, 
                                   optimizer = optim.Adagrad, max_epochs = 50, batch_size = 10)
model.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=cnn_maker(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (conv3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (fc1): Linear(in_features=59904, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=32, bias=True)
    (drop): Dropout(p=0.5, inplace=False)
    (fc_f): Linear(in_features=32, out_features=1, bias=True)
    (prob): Sigmoid()
  ),
)

In [28]:
param_grid = {
    'module__weight_init': [init.uniform_, init.normal_, init.zeros_,
                           init.xavier_normal_, init.xavier_uniform_,
                           init.kaiming_normal_, init.kaiming_uniform_]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_full[:100].astype(np.float32), y_full[:100].reshape(-1, 1).astype(np.float32))
 
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1       55.0840       0.4000       60.0000  4.2354
      2       22.0135       0.6000        0.6813  4.1717
      3        1.1806       0.6000        0.6711  4.2079
      4        0.9975       0.4000        0.6983  4.2113
      5        0.6998       0.4000        0.6945  4.1852
      6        0.6944       0.6000        0.6926  4.1671
      7        0.6820       0.6000        0.6740  4.1660
      8        0.7200       0.6000        0.6713  4.1264
      9        0.7092       0.6000        0.6827  4.1715
     10        0.6940       0.6000        0.6865  4.1184
     11        0.7098       0.6000        0.6847  4.1556
     12        0.6630       0.6000        0.6676  4.0959
     13        0.7069       0.6000        0.6769  4.1401
     14        0.6741       0.6000        0.6790  4.1899
     15        0.6845       0.6000        0.6706  4.1762
     16        0.6849       0.6

# Best: 0.629531 using {'module__weight_init': <function xavier_normal_ at 0x000001E68E0345E0>}

# Not sure about the above, but we have some nice stuff to go off. Let's put it all together in the other notebook and get a result